# 作業2

In [85]:
import pandas as pd
import datetime as dt
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from plotly import plotly
from plotly import graph_objs
import pickle
import plotly.tools as tls
tls.set_credentials_file(username='ericmina83', api_key='lk2sUQGX2dOg9Y8i84w2')

In [28]:
data = pd.read_csv('./LargeTrain.csv')
     
# get predictors and target titles
target ='Class'
predictors = [x for x in data.columns if x not in [target]]

# prepare training data and testing data
mask = np.random.rand(len(data)) < 0.8 
training = data[mask]
test = data[~mask]

# prepare training predictors and target
X = training[predictors]
y = training[target]

# initialize model
xgb_model = XGBClassifier(
    learning_rate = 0.1,
    n_estimators = 200,
    max_depth = 3,
    min_child_weight = 3,
    gamma = 0,
    subsample = 0.8,
    objective= 'binary:logistic',
    colsample_bytree = 0.8,
    nthread = 4,
    scale_pos_weight = 1,
    seed = 27)

## Step 1: 尋找 最佳的 max_depth 和 min_child_weight

In [7]:
param_test1 = {
    'max_depth':list(range(1,11,2)),
    'min_child_weight':list(range(1,11,2))  
}

gsearch1 = GridSearchCV(
    estimator = xgb_model,
    param_grid = param_test1,
    n_jobs = 4,
    iid = False,
    cv = 5)

gsearch1.fit(X, y)
gsearch1.grid_scores_

C:\Users\Eric\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.99687, std: 0.00054, params: {'min_child_weight': 3, 'max_depth': 3},
 mean: 0.99650, std: 0.00111, params: {'min_child_weight': 5, 'max_depth': 3},
 mean: 0.99650, std: 0.00161, params: {'min_child_weight': 7, 'max_depth': 3},
 mean: 0.99669, std: 0.00098, params: {'min_child_weight': 3, 'max_depth': 5},
 mean: 0.99660, std: 0.00138, params: {'min_child_weight': 5, 'max_depth': 5},
 mean: 0.99687, std: 0.00114, params: {'min_child_weight': 7, 'max_depth': 5},
 mean: 0.99696, std: 0.00095, params: {'min_child_weight': 3, 'max_depth': 7},
 mean: 0.99696, std: 0.00099, params: {'min_child_weight': 5, 'max_depth': 7},
 mean: 0.99678, std: 0.00142, params: {'min_child_weight': 7, 'max_depth': 7}]

In [13]:
print("best params:", gsearch1.best_params_, "best score: ", gsearch1.best_score_)

best params: {'min_child_weight': 5, 'max_depth': 7} best score:  0.996964285406


## Step 2: 尋找 最佳的 gamma

In [14]:
# replace the best model to xgb_model
xgb_model = gsearch1.best_estimator_

param_test2 = {
 'gamma':[i/10.0 for i in range(0,5)]
}

gsearch2 = GridSearchCV(
    estimator = xgb_model,
    param_grid = param_test2,
    n_jobs = 4,
    iid = False, v 
    cv = 5)

gsearch2.fit(X, y)
gsearch2.grid_scores_

C:\Users\Eric\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.99696, std: 0.00099, params: {'gamma': 0.0},
 mean: 0.99678, std: 0.00096, params: {'gamma': 0.1},
 mean: 0.99669, std: 0.00118, params: {'gamma': 0.2},
 mean: 0.99687, std: 0.00102, params: {'gamma': 0.3},
 mean: 0.99650, std: 0.00111, params: {'gamma': 0.4}]

## Step3: 尋找 最佳的 subsample 與 colsample_bytree

In [16]:
xgb_model = gsearch2.best_estimator_
param_test4 = {
     'subsample':[i/10.0 for i in range(6,10)],
     'colsample_bytree':[i/10.0 for i in range(6,10)]
}

gsearch4 = GridSearchCV(
    estimator = xgb_model,
    param_grid = param_test4,
    n_jobs = 4,
    iid = False,
    cv = 5)

gsearch4.fit(X, y)
gsearch4.grid_scores_

C:\Users\Eric\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.99632, std: 0.00113, params: {'colsample_bytree': 0.6, 'subsample': 0.6},
 mean: 0.99678, std: 0.00167, params: {'colsample_bytree': 0.6, 'subsample': 0.7},
 mean: 0.99678, std: 0.00130, params: {'colsample_bytree': 0.6, 'subsample': 0.8},
 mean: 0.99687, std: 0.00135, params: {'colsample_bytree': 0.6, 'subsample': 0.9},
 mean: 0.99641, std: 0.00134, params: {'colsample_bytree': 0.7, 'subsample': 0.6},
 mean: 0.99669, std: 0.00134, params: {'colsample_bytree': 0.7, 'subsample': 0.7},
 mean: 0.99660, std: 0.00107, params: {'colsample_bytree': 0.7, 'subsample': 0.8},
 mean: 0.99660, std: 0.00115, params: {'colsample_bytree': 0.7, 'subsample': 0.9},
 mean: 0.99696, std: 0.00132, params: {'colsample_bytree': 0.8, 'subsample': 0.6},
 mean: 0.99696, std: 0.00095, params: {'colsample_bytree': 0.8, 'subsample': 0.7},
 mean: 0.99696, std: 0.00099, params: {'colsample_bytree': 0.8, 'subsample': 0.8},
 mean: 0.99660, std: 0.00090, params: {'colsample_bytree': 0.8, 'subsample': 0.9},
 mea

In [19]:
xgb_model = gsearch4.best_estimator_
param_test6 = {
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}

gsearch6 = GridSearchCV(
    estimator = xgb_model,
    param_grid = param_test6,
    n_jobs = 4,
    iid = False,
    cv = 5)

gsearch6.fit(X, y)
gsearch6.grid_scores_

C:\Users\Eric\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.99696, std: 0.00099, params: {'reg_alpha': 1e-05},
 mean: 0.99696, std: 0.00107, params: {'reg_alpha': 0.01},
 mean: 0.99696, std: 0.00085, params: {'reg_alpha': 0.1},
 mean: 0.99678, std: 0.00120, params: {'reg_alpha': 1},
 mean: 0.98758, std: 0.00274, params: {'reg_alpha': 100}]

In [27]:
xgb_model.score

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.0, learning_rate=0.1, max_delta_step=0, max_depth=7,
       min_child_weight=5, missing=None, n_estimators=250, nthread=4,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.8)

{'type': 'heatmap'}